## Arxiv--Research

In [ ]:

## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

* WikipediaQueryRun uses WikipediaAPIWrapper 
* WikipediaAPIWrapper → knows the real HTTP request format, API endpoints, and how to parse the response.

In [ ]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

# So now wiki is a  built in tool that can be used to query Wikipedia
wiki.name

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

# So now arxiv is a built in tool that can be used to query Arxiv
print(arxiv.name)

In [ ]:
# Now we can use these tools in our agents
tools=[wiki,arxiv]

## Creating Custom tools[eg: RAG Tool]

In [ ]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

In [ ]:
# Convert the retriever into a tool
from langchain.tools.retriever import create_retriever_tool

# Here langsmith-search is the name of the tool and the description is a short description of what the tool does
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ") 

retriever_tool.name

In [ ]:
tools=[wiki,arxiv,retriever_tool]

In [ ]:
tools

## Run all this tools with Agents and LLM Models

In [ ]:
## Tools, LLM-->AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [ ]:
## Prompt Template

# Hub is a place where you can find prompts and agents created by the community

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
## Agents

# Combine the tools and the prompt to create an agent
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)
agent

In [ ]:
## Agent Executer is the class that will execute the agent with the tools and the prompt


from langchain.agents import AgentExecutor

# verbose=True will print the steps taken by the agent
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})

In [ ]:
agent_executor.invoke({"input":"What is machine learning"})

In [ ]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})